# Подготовка данных для обучения моделей

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* https://scikit-learn.org/stable/modules/compose.html#pipeline-chaining-estimators
* https://pytorch.org/docs/stable/data.html
* https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann


## Задачи для совместного разбора

1. Рассмотрите, как можно выстраивать конвейер предобработки данных при помощи `Pipeline` из `sklearn`

2. Создайте синтетический датасет для задачи регрессии и представьте его в виде `torch.utils.data.Dataset`

## Задачи для самостоятельного решения

In [237]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler

<p class="task" id="1"></p>

1\. Считайте файл `bank-full.csv` ([источник](https://www.kaggle.com/datasets/hariharanpavan/bank-marketing-dataset-analysis-classification)) в виде `pd.DataFrame`. Используя `Pipeline` из `sklearn`, закодируйте значения в нечисловых столбцах целыми числами, после чего нормализуйте получившиеся признаки. Выведите преобразованные данные на экран.

In [238]:
df = pd.read_csv("bank-full.csv")

num_col = df.select_dtypes(include=["int64", "float64"]).columns
cat_col = df.select_dtypes(include=["object"]).columns


pipline = Pipeline([
    ("encoder", OrdinalEncoder()),
    ("scaler", MinMaxScaler())
])
df_trnsf = pipline.fit_transform(df)
df_trnsf = pd.DataFrame(df_trnsf, columns=df.columns)

df_trnsf.head()


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,0.526316,0.363636,0.5,0.666667,0.0,0.423608,1.0,0.0,1.0,0.133333,0.727273,0.166031,0.0,0.0,0.0,1.0,0.0
1,0.342105,0.818182,1.0,0.333333,0.0,0.131854,1.0,0.0,1.0,0.133333,0.727273,0.096056,0.0,0.0,0.0,1.0,0.0
2,0.197368,0.181818,0.5,0.333333,0.0,0.128087,1.0,1.0,1.0,0.133333,0.727273,0.048346,0.0,0.0,0.0,1.0,0.0
3,0.381579,0.090909,0.5,1.000000,0.0,0.337659,1.0,0.0,1.0,0.133333,0.727273,0.058524,0.0,0.0,0.0,1.0,0.0
4,0.197368,1.000000,1.0,1.000000,0.0,0.127948,0.0,0.0,1.0,0.133333,0.727273,0.125954,0.0,0.0,0.0,1.0,0.0


<p class="task" id="2"></p>

2\. Опишите класс `BankDatasetBase`. Решение должно удовлетворять следующим критериям:

* класс наследуется от `torch.utils.data.Dataset`;
* при создании объекта в конструктор передается набор данных в виде `pd.DataFrame`;
* объекты класса имеют поля `X` и `y` с признаками и метками соответственно;
* класс реализует интерфейс последовательностей (`__getitem__` + `__len__`);
* `obj[i]` возвращает кортеж, содержащий `i`-ую строку из `obj.X` и `i`-ую строку из `obj.y`.
    
Создайте объект класса `BankDatasetBase` и продемонстрируйте работоспособность.

In [239]:
import torch
from torch.utils.data import Dataset

In [240]:
class BankDatasetBase(Dataset):
    def __init__(self, data: pd.DataFrame) -> None:

      self.X = data.iloc[:, :-1].values
      self.y = data.iloc[:, -1].values

    def __getitem__(self, idx) -> tuple:

        return self.X[idx], self.y[idx]

    def __len__(self) -> int:

        return len(self.X)

if __name__ == "__main__":
  dataset = BankDatasetBase(df)
  print(f"Длина набора данных: {len(dataset)}")
  print(f"Первый элемент: {dataset[0]}")
  print(f"Последний элемент: {dataset[-1]}")

Длина набора данных: 45211
Первый элемент: (array([58, 'management', 'married', 'tertiary', 'no', 2143, 'yes', 'no',
       'unknown', 5, 'may', 261, 1, -1, 0, 'unknown'], dtype=object), 'no')
Последний элемент: (array([37, 'entrepreneur', 'married', 'secondary', 'no', 2971, 'no', 'no',
       'cellular', 17, 'nov', 361, 2, 188, 11, 'other'], dtype=object), 'no')


<p class="task" id="3"></p>

3\. Опишите класс `BankDataset`. Решение должно удовлетворять всем критериям из предыдущего задания, а также:
* при создании объекта в конструктор может быть передан необязательный аргумент `transform: callable`;
* если данный аргумент был передан, то при получении `i`-го элемента, нужно вызвать `transform(x, y)` и вернуть полученный результат.

Создайте объект класса `BankDataset` и продемонстрируйте работоспособность.

In [243]:
class BankDataset:
    def __init__(self, data: pd.DataFrame, transform: callable = None) -> None:
        self.data = data
        self.transform = transform

    def __getitem__(self, idx) -> tuple:
        row = self.data.iloc[idx]
        x = row.iloc[:-1]
        y = row.iloc[-1]

        if self.transform is not None:
            x, y = self.transform(x, y)

        return x, y

    def __len__(self) -> int:
        return len(self.data)


if __name__ == "__main__":

  def ex_transform(x, y):

    y = 1 if y=="yes" else 0
    return x, y
  dataset_ntr = BankDataset(df)
  print(f"Без transform \n{dataset_ntr[0]}")
  dataset_wtr = BankDataset(df, transform=ex_transform)
  print(f"С transform \n{dataset_wtr[0]}")


Без transform 
(age                  58
job          management
marital         married
education      tertiary
default              no
balance            2143
housing             yes
loan                 no
contact         unknown
day                   5
month               may
duration            261
campaign              1
pdays                -1
previous              0
poutcome        unknown
Name: 0, dtype: object, 'no')
С transform 
(age                  58
job          management
marital         married
education      tertiary
default              no
balance            2143
housing             yes
loan                 no
contact         unknown
day                   5
month               may
duration            261
campaign              1
pdays                -1
previous              0
poutcome        unknown
Name: 0, dtype: object, 0)


<p class="task" id="4"></p>

4\. Опишите класс `LabelEncoderTransform`. Решение должно удовлетворять следующим критериям:

* при создании объекта в конструктор передаются названия нечисловых столбцов в датасете
* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` имеет два параметра (признаки и класс) и возвращает кортеж, состоящих двух элементов: набора признаков, в котором нечисловые характеристики закодированы целыми числами; и чисел - меток классов;
* состояние объекта (индексы для кодирования) обновляется в момент очередного вызова `__call__` (т.е. все данные сразу никогда не передаются никакому методу объекта).
    
Продемонстрируйте работоспособность, создав объект `BankDataset` и передав при создании объект класса `LabelEncoderTransform`.

In [246]:
class LabelEncoderTransform:
    def __init__(self, category_columns: list[str]) -> None:
        self.category_columns = category_columns
        self.encoders = {}

    def __call__(self, x, y) -> tuple:

        transformed_x = x.copy()

        for col in self.category_columns:
            value = x[col]

            if col not in self.encoders:
                self.encoders[col] = {}

            encoder = self.encoders[col]

            if value not in encoder:
                encoder[value] = len(encoder)

            transformed_x[col] = encoder[value]

        y = 1 if y == "yes" else 0

        return tuple(transformed_x), y



if __name__ == "__main__":

  cat_col = [
        "job", "marital", "education", "default",
        "housing", "loan", "contact", "month", "poutcome"
    ]

  transform = LabelEncoderTransform(category_columns=cat_col)
  dataset = BankDataset(df, transform=transform)

  print("Пример необработанной строки:")
  print(df.iloc[0])

  print("\nТрансформированная строка:")
  print(dataset[0])

  print("Пример необработанной строки:")
  print(df.iloc[-1])

  print("\nТрансформированная строка:")
  print(dataset[-1])

  print("Пример необработанной строки:")
  print(df.iloc[100])

  print("\nТрансформированная строка:")
  print(dataset[100])


Пример необработанной строки:
age                  58
job          management
marital         married
education      tertiary
default              no
balance            2143
housing             yes
loan                 no
contact         unknown
day                   5
month               may
duration            261
campaign              1
pdays                -1
previous              0
poutcome        unknown
y                    no
Name: 0, dtype: object

Трансформированная строка:
((np.int64(58), 0, 0, 0, 0, np.int64(2143), 0, 0, 0, np.int64(5), 0, np.int64(261), np.int64(1), np.int64(-1), np.int64(0), 0), 0)
Пример необработанной строки:
age                    37
job          entrepreneur
marital           married
education       secondary
default                no
balance              2971
housing                no
loan                   no
contact          cellular
day                    17
month                 nov
duration              361
campaign                2
pdays       

<p class="task" id="5"></p>

5\. Опишите класс `ToTensor`.  Решение должно удовлетворять следующим критериям:
* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` принимает имеет параметра (признаки и класс в числовом виде) и возвращает кортеж, состоящих 2 тензоров.

Опишите класс `Compose`.  Решение должно удовлетворять следующим критериям:
* при создании объекта в конструктор передается список объектов `transforms`, каждый из которых имеет метод `__call__(x, y)`;
* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` принимает имеет параметра (признаки и класс в числовом виде) и и возвращает кортеж, полученный путем последовательного вызова объектов из `transforms`.

Продемонстрируйте работоспособность, создав объект `BankDataset` и передав при создании список из объектов LabelEncoderTransform и ToTensor.

In [247]:
class ToTensor(object):
    def __call__(self, X, y) -> tuple:
        X_tensor = torch.tensor(X)
        y_tensor = torch.tensor(y)
        return X_tensor, y_tensor

class Compose(object):
    def __init__(self, transforms: list) -> None:
        self.transforms = transforms

    def __call__(self, X, y):
        for transform in self.transforms:
          X, y = transform(X, y)
        return X, y

if __name__ == "__main__":
  transform = Compose([LabelEncoderTransform(category_columns=cat_col), ToTensor()])
  dataset = BankDataset(df, transform=transform)
  x0, y0 = dataset[0]
  print(x0)
  print(y0)
  x01, y01 = dataset[-1]
  print(x01)
  print(y01)



tensor([  58,    0,    0,    0,    0, 2143,    0,    0,    0,    5,    0,  261,
           1,   -1,    0,    0])
tensor(0)
tensor([  37,    1,    0,    1,    0, 2971,    1,    0,    1,   17,    1,  361,
           2,  188,   11,    1])
tensor(0)


<p class="task" id="6"></p>

6\. Разделите датасет из предыдущего задания на обучающую и тестовую выборку в соотношении 75% на 25%. Создайте объект `DataLoader` для получения пакетов размера 64, полученных из перемешанного обучающего датасета. Кастомизируйте `DataLoader` таким образом, чтобы пакет признаков был представлен в виде трехмерного тензора размера 64x2x8 (разделите 16 признаков на два тензора по 8). Получите один пакет и выведите на экран размерность тензоров пакета.

In [248]:
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

In [250]:
class ToTensor:
    def __call__(self, x, y) -> tuple:
        x_tensor = torch.tensor(x)
        x_tensor = x_tensor.view(2, 8)
        y_tensor = torch.tensor(y)
        return x_tensor, y_tensor



df_train, df_test = train_test_split(df, test_size=0.25, random_state=42)


transform = Compose([
    LabelEncoderTransform(category_columns=cat_col),
    ToTensor()
])


train_dataset = BankDataset(df_train, transform=transform)
test_dataset = BankDataset(df_test, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)


X_batch, y_batch = next(iter(train_loader))

print("Размерность признаков:", X_batch.shape)
print("Размерность меток:", y_batch.shape)



Размерность признаков: torch.Size([64, 2, 8])
Размерность меток: torch.Size([64])


## Обратная связь
- [ ] Хочу получить обратную связь по решению